# Problemas de Gradientes Desvanecentes/Explodindo e Soluções para Redes Neurais Profundas

Esse capítulo explora os desafios de treinar redes neurais profundas, especificamente os problemas de gradientes desvanecentes e explodindo, e apresenta diversas técnicas para superá-los, bem como outras estratégias para otimizar o treinamento e evitar o overfitting.


## 1. Problemas de Gradientes Desvanecentes e Explodindo

Ao treinar redes neurais profundas com o algoritmo de backpropagation, os gradientes (que indicam a direção e a magnitude do ajuste dos pesos) podem se tornar muito pequenos ou muito grandes à medida que são propagados pelas camadas.

* **Gradientes Desvanecentes:** Os gradientes ficam cada vez menores à medida que descem para as camadas inferiores da rede. Isso faz com que os pesos das camadas mais baixas sejam atualizados minimamente, impedindo que a rede aprenda efetivamente e que o treinamento convirja para uma boa solução. 

* **Gradientes Explodindo:** O oposto também pode acontecer, onde os gradientes crescem descontroladamente, levando a atualizações de peso insanamente grandes. Isso faz com que o algoritmo divirja. É mais comum em Redes Neurais Recorrentes (RNNs).

**Causas:** Xavier Glorot e Yoshua Bengio (2010) mostraram que a combinação da função de ativação sigmoide logística (popular na época) e a inicialização de pesos aleatória (distribuição normal com média 0 e desvio padrão 1) contribuíam para esses problemas. A sigmoide satura para valores de entrada grandes (positivos ou negativos), onde sua derivada é quase zero, impedindo a propagação eficaz dos gradientes.


## 2. Inicialização de Pesos (Glorot e He)

Para mitigar os problemas de gradientes, é crucial garantir que o sinal flua corretamente em ambas as direções (forward e backward).

* **Princípio:** A variância das saídas de cada camada deve ser igual à variância de suas entradas, e os gradientes devem ter variância igual antes e depois de fluir por uma camada na direção inversa.
* **Inicialização Glorot (Xavier):** Proposta por Glorot e Bengio, essa estratégia inicializa os pesos de forma aleatória com uma distribuição normal (média 0) ou uniforme. 
* **Inicialização He:** Similar à Glorot, mas otimizada para a função de ativação ReLU e suas variantes.
* **Inicialização LeCun:** Proposta por Yann LeCun, é equivalente à Glorot e é recomendada para a função de ativação SELU.

**Keras:** Por padrão, o Keras usa a inicialização Glorot com distribuição uniforme. Para mudar pode usar `kernel_initializer="he_normal"` ou `kernel_initializer="he_uniform"` ao criar camadas `Dense`.


## 3. Funções de Ativação Não-Saturantes

A escolha da função de ativação é fundamental para evitar a saturação e, consequentemente, os problemas de gradientes.

* **ReLU (Rectified Linear Unit):** Mais rápida de computar e não satura para valores positivos. No entanto, sofre do problema de "dying ReLUs", onde alguns neurônios param de produzir qualquer coisa além de 0 se a soma ponderada de suas entradas for sempre negativa.
* **Leaky ReLU:** Define a função como $\text{max}(\alpha z, z)$, onde $\alpha$ é uma pequena inclinação para $z < 0$ (tipicamente 0.01). Isso evita que os neurônios "morram". Variantes incluem RReLU ( $\alpha$ aleatório) e PReLU ( $\alpha$ aprende durante o treinamento).
* **ELU (Exponential Linear Unit):** Definida como $\alpha(\text{exp}(z) - 1)$ para $z < 0$ e $z$ para $z \geq 0$. Supera as variantes da ReLU em muitos experimentos, reduzindo o tempo de treinamento e melhorando o desempenho. Possui valores negativos (ajudando a ter média mais próxima de zero) e um gradiente não nulo para $z < 0$, evitando neurônios mortos. É suave em $z = 0$, o que acelera o Gradiente Descendente. O custo computacional é maior devido à função exponencial.
* **SELU (Scaled Exponential Linear Unit):** Uma versão escalada da ELU. Se a rede for composta exclusivamente por camadas densas e as entradas forem padronizadas (média 0, desvio padrão 1), e os pesos inicializados com LeCun normal, a rede se "auto-normaliza". Isso significa que a saída de cada camada tende a preservar média 0 e desvio padrão 1, resolvendo os problemas de gradientes e superando outras funções de ativação em redes neurais densas e profundas.


## 4. Normalização em Lotes (Batch Normalization)

* **Funcionamento:** Adiciona uma operação antes ou depois da função de ativação de cada camada oculta. Essa operação centraliza os dados (média 0), normaliza-os (desvio padrão 1), e depois os escala e desloca usando dois novos parâmetros aprendíveis ($\gamma$ para escala e $\beta$ para deslocamento). Isso permite que a rede aprenda a escala e a média ótimas para as entradas de cada camada.
* **Benefícios:**
    * Reduz drasticamente os problemas de gradientes desvanecentes/explodindo, permitindo o uso de funções de ativação saturantes como Tanh.
    * Torna as redes menos sensíveis à inicialização de pesos.
    * Permite o uso de taxas de aprendizado maiores, acelerando o treinamento.
    * Atua como um regularizador, reduzindo a necessidade de outras técnicas de regularização.
* **Keras:** Adicionar uma camada `keras.layers.BatchNormalization()` antes ou depois de cada camada oculta. 


## 5. Recorte de Gradiente (Gradient Clipping)

Uma técnica para mitigar o problema de gradientes explodindo, especialmente em Redes Neurais Recorrentes (RNNs).

* **Funcionamento:** Simplesmente recorta os gradientes durante o backpropagation para que nunca excedam um certo limite.
* **Tipos:**
    * **Recorte por Valor (`clipvalue`):** Limita cada componente do vetor gradiente a um valor entre -threshold e +threshold. Pode alterar a direção do vetor gradiente.
    * **Recorte por Norma (`clipnorm`):** Limita a norma $\ell_2$ de todo o vetor gradiente se for maior que o threshold. Preserva a orientação do vetor gradiente.
* **Keras:** Defina `clipvalue` ou `clipnorm` ao criar um otimizador: `optimizer = keras.optimizers.SGD(clipvalue=1.0)`.


## 6. Reutilização de Camadas Pré-treinadas (Transfer Learning)

Uma boa estratégia para acelerar o treinamento e reduzir a necessidade de grandes volumes de dados de treinamento, especialmente em tarefas semelhantes.

* **Funcionamento:** Reutiliza as camadas inferiores de uma rede neural pré-treinada em uma tarefa similar. As camadas inferiores aprendem características de baixo nível (ex: bordas, texturas em imagens) que são frequentemente úteis em várias tarefas.
* **Processo:**
    1.  Carregue o modelo pré-treinado.
    2.  Crie um novo modelo usando as camadas do modelo pré-treinado (excluindo a camada de saída).
    3.  Adicione uma nova camada de saída (e possivelmente algumas camadas ocultas superiores).
    4.  **Congele** as camadas reutilizadas (defina `layer.trainable = False`) e treine apenas as novas camadas por algumas épocas.
    5.  **Descongele** as camadas reutilizadas (defina `layer.trainable = True`) e continue o treinamento com uma taxa de aprendizado menor para "ajustar" os pesos para a nova tarefa.
* **Keras:** Para clonar um modelo e copiar seus pesos, usar `keras.models.clone_model()` e `model.set_weights()`.


## 7. Pré-treinamento Não Supervisionado e em Tarefas Auxiliares

Quando não há muitos dados rotulados e nenhum modelo pré-treinado similar disponível.

* **Pré-treinamento Não Supervisionado:** Treinar as camadas da rede uma a uma, usando algoritmos de detecção de características não supervisionados (ex: Máquinas de Boltzmann Restritas, Autoencoders). Após o pré-treinamento de todas as camadas, adiciona-se a camada de saída e realiza-se o ajuste fino com os poucos dados rotulados.
* **Pré-treinamento em Tarefa Auxiliar:** Treinar uma primeira rede neural em uma tarefa auxiliar para a qual é fácil obter ou gerar dados rotulados. As camadas inferiores dessa rede aprenderão detectores de características que serão reutilizáveis para a tarefa principal. Ex: Treinar uma rede para detectar se duas imagens são da mesma pessoa para depois usá-la para reconhecimento facial com poucos dados.
* **Self-supervised learning:** Um tipo de aprendizado não supervisionado onde os rótulos são gerados automaticamente a partir dos próprios dados.


## 8. Otimizadores Mais Rápidos

O uso de otimizadores avançados pode acelerar significativamente o treinamento.

* **Otimização por Momento (Momentum Optimization):** Inspirado por uma bola de boliche descendo uma encosta. Além do gradiente local, considera os gradientes anteriores para construir um "momento" que acelera o treinamento nas direções consistentes e ajuda a escapar de plateaus.
    * Keras: `keras.optimizers.SGD(lr=0.001, momentum=0.9)`
* **Gradiente Acelerado de Nesterov (NAG):** Uma pequena variação do Momentum que mede o gradiente da função de custo um pouco à frente na direção do momento. Geralmente mais rápido e com menos oscilação que o Momentum vanilla.
    * Keras: `keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)`
* **AdaGrad:** Escala o vetor gradiente de forma adaptativa, diminuindo a taxa de aprendizado mais rapidamente para dimensões mais íngremes. Ajuda a apontar as atualizações mais diretamente para o ótimo global. No entanto, tende a parar muito cedo em redes neurais profundas.
* **RMSProp:** Corrige a desvantagem do AdaGrad ao acumular apenas os gradientes das iterações mais recentes, usando decaimento exponencial. Geralmente tem um desempenho muito melhor que o AdaGrad.
    * Keras: `keras.optimizers.RMSprop(lr=0.001, rho=0.9)`
* **Adam (Adaptive Moment Estimation):** Combina as ideias de Momentum e RMSProp, mantendo uma média exponencialmente decrescente de gradientes passados e uma média exponencialmente decrescente de gradientes quadrados passados. É um algoritmo de taxa de aprendizado adaptativa, exigindo menos ajuste da taxa de aprendizado.
    * Keras: `keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)`
* **Nadam:** Muitas vezes converge ligeiramente mais rápido que Adam.

**Consideração:** Embora os otimizadores adaptativos (RMSProp, Adam, Nadam) sejam rápidos, um estudo de 2017 por Wilson et al. mostrou que eles podem levar a soluções que generalizam mal em alguns conjuntos de dados. Nesses casos, o Gradiente Acelerado de Nesterov pode ser uma alternativa melhor.


## 9. Agendamento da Taxa de Aprendizado (Learning Rate Scheduling)

Variar a taxa de aprendizado durante o treinamento pode acelerar a convergência e levar a uma solução melhor.

* **Agendamento de Potência (Power Scheduling):** Taxa de aprendizado decai com o número da iteração ($\eta(t) = \eta_0 / (1 + t/k)^c$).
* **Agendamento Exponencial (Exponential Scheduling):** Taxa de aprendizado decai exponencialmente ($\eta(t) = \eta_0 \cdot 0.1^{t/s}$).
* **Agendamento Constante por Partes (Piecewise Constant Scheduling):** Usa uma taxa de aprendizado constante por um número de épocas, depois uma taxa menor por outro número de épocas, e assim por diante.
* **Agendamento por Desempenho (Performance Scheduling):** Reduz a taxa de aprendizado quando o erro de validação para de cair.


## 10. Evitando Overfitting Através da Regularização

Redes neurais profundas com muitos parâmetros são propensas a overfitting.

* **Early Stopping:** Interromper o treinamento quando o desempenho no conjunto de validação começa a piorar. 
* **Batch Normalization:** Embora projetada para problemas de gradientes, atua também como um bom regularizador.
* **Regularização $\ell_1$ e $\ell_2$:** Adiciona um termo de penalidade à função de custo, forçando os pesos a serem menores (ou zero no caso de $\ell_1$).
    * Keras: Use `kernel_regularizer=keras.regularizers.l1(0.01)` ou `l2(0.01)` ou `l1_l2()`.
* **Dropout:** Em cada passo de treinamento, um percentual `p` de neurônios é temporariamente "desativado". Isso impede que os neurônios co-adaptem e força a rede a ser mais robusta, generalizando melhor. Durante a inferência, o dropout é desativado e os pesos são escalados pela probabilidade de permanência (`1-p`) ou as saídas dos neurônios são divididas por `(1-p)` durante o treinamento.
    * Keras: `keras.layers.Dropout(rate=0.2)`